# Dense Neural Network (NN) Exploration

In this notebook, we will lay out our approach for a NN approach to predict a pose based on landmark.

In [1]:
import numpy as np

In [16]:
array = ([[[[0.58768964, 0.36290908, 0.603793  ],
         [0.5932051 , 0.3443019 , 0.60789067],
         [0.59303284, 0.3484372 , 0.7001313 ],
         [0.5522757 , 0.31307817, 0.5474181 ],
         [0.5549346 , 0.3191042 , 0.6794196 ],
         [0.48681545, 0.34274042, 0.74582547],
         [0.48455274, 0.3486129 , 0.67955416],
         [0.5900916 , 0.23778538, 0.67808187],
         [0.56829524, 0.23839937, 0.7717802 ],
         [0.6503178 , 0.11018609, 0.67483616],
         [0.6161561 , 0.14426906, 0.4420369 ],
         [0.29321983, 0.5450648 , 0.844166  ],
         [0.30226567, 0.54754007, 0.8217478 ],
         [0.45619473, 0.6819893 , 0.5754474 ],
         [0.46339905, 0.66756546, 0.7072478 ],
         [0.59998393, 0.8396907 , 0.6530191 ],
         [0.5962424 , 0.7989042 , 0.7743349 ]]]])
lm = array[0][0]

In [17]:
lm

[[0.58768964, 0.36290908, 0.603793],
 [0.5932051, 0.3443019, 0.60789067],
 [0.59303284, 0.3484372, 0.7001313],
 [0.5522757, 0.31307817, 0.5474181],
 [0.5549346, 0.3191042, 0.6794196],
 [0.48681545, 0.34274042, 0.74582547],
 [0.48455274, 0.3486129, 0.67955416],
 [0.5900916, 0.23778538, 0.67808187],
 [0.56829524, 0.23839937, 0.7717802],
 [0.6503178, 0.11018609, 0.67483616],
 [0.6161561, 0.14426906, 0.4420369],
 [0.29321983, 0.5450648, 0.844166],
 [0.30226567, 0.54754007, 0.8217478],
 [0.45619473, 0.6819893, 0.5754474],
 [0.46339905, 0.66756546, 0.7072478],
 [0.59998393, 0.8396907, 0.6530191],
 [0.5962424, 0.7989042, 0.7743349]]

In [18]:
lm[0][1]

0.36290908

In [ ]:
KEYPOINT_DICT = {
    ‘nose’: 0,
    ‘left_eye’: 1,
    ‘right_eye’: 2,
    ‘left_ear’: 3,
    ‘right_ear’: 4,
    ‘left_shoulder’: 5,
    ‘right_shoulder’: 6,
    ‘left_elbow’: 7,
    ‘right_elbow’: 8,
    ‘left_wrist’: 9,
    ‘right_wrist’: 10,
    ‘left_hip’: 11,
    ‘right_hip’: 12,
    ‘left_knee’: 13,
    ‘right_knee’: 14,
    ‘left_ankle’: 15,
    ‘right_ankle’: 16
}

In [30]:
lm[0]

[0.58768964, 0.36290908, 0.603793]

In [54]:
landmark_dict = {
    'landmarks_left_elbow' : (lm[5], lm[7], lm[9]),
    'landmarks_right_elbow' : (lm[6], lm[8], lm[10]),
    'landmarks_left_shoulder' : (lm[7], lm[9], lm[11]),
    'landmarks_right_shoulder' : (lm[8], lm[10], lm[12]),
    'landmarks_hip_left' : (lm[5], lm[11], lm[13]),
    'landmarks_hip_right' : (lm[6], lm[12], lm[14]),
    'landmarks_left_knee' : (lm[11], lm[13], lm[15]),
    'landmarks_right_knee' : (lm[12], lm[14], lm[16])
        }

In [56]:
def angle_calculator(landmarks):
    
    # Converting the points into numpy arrays
    p1 = np.array(landmarks[0])
    p2 = np.array(landmarks[1])
    p3 = np.array(landmarks[2])
    
    # Creating the Vectors between two points
    vec_p1 = p1-p2
    vec_p2 = p3-p2
    
    # Calculating the cosine of the angle
    cosine_angle = np.dot(vec_p1,vec_p2) / (np.linalg.norm(vec_p1)*np.linalg.norm(vec_p2))
    
    # Calculating angle
    angle = np.arccos(cosine_angle)
    
    # Calculating degrees
    angle_degrees = np.degrees(angle)
    
    return angle_degrees
    
    
    

In [57]:
angle_left_elbow = angle_calculator(landmark_dict['landmarks_left_elbow'])
angle_left_elbow

150.10594644117597

In [58]:
angle_name_list = ['angle_elbow_left',
                   'angle_elbow_right',
                   'angle_shoulder_left',
                   'angle_shoulder_right',
                   'angle_hip_left',
                   'angle_hip_right',
                   'angle_knee_left',
                   'angle_knee_right',
                  ]

In [61]:
angle_dict = {}

In [62]:
for dictkey, listkey in zip(landmark_dict, angle_name_list):
    i = angle_calculator(landmark_dict[dictkey])
    angle_dict[listkey] = i

In [63]:
angle_dict

{'angle_elbow_left': 150.10594644117597,
 'angle_elbow_right': 73.83993477914649,
 'angle_shoulder_left': 20.769064686404477,
 'angle_shoulder_right': 36.064848022681005,
 'angle_hip_left': 72.6893113496775,
 'angle_hip_right': 72.01545656467914,
 'angle_knee_left': 108.09862487255467,
 'angle_knee_right': 129.9717250242498}

These are the angles we need!

In [64]:
angle_list = [angle_dict[key] for key in angle_dict]

In [65]:
angle_list

[150.10594644117597,
 73.83993477914649,
 20.769064686404477,
 36.064848022681005,
 72.6893113496775,
 72.01545656467914,
 108.09862487255467,
 129.9717250242498]